## Examples

In [9]:
!python -m pip install numpy

In [14]:
!python -m pip uninstall -y changepoint_online

3087.27s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Found existing installation: changepoint-online 0.0.1
Uninstalling changepoint-online-0.0.1:
  Successfully uninstalled changepoint-online-0.0.1


In [15]:
!python -m pip install '../package/'

3094.72s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Processing /home/romano/Dropbox/changepoint.online/python/package
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for changepoint-online: filename=changepoint_online-0.0.1-py3-none-any.whl size=18424 sha256=ece2d9f12150fb499dc69121d0620ea8c338634ff82d12862a2cfb29cca7644a
  Stored in directory: /tmp/pip-ephem-wheel-cache-t4dl9edy/wheels/ef/98/3a/e389858773e4fe5f47ec0415eb44c05e2319de69019c54a854
Successfully built changepoint-online


In [1]:
from changepoint_online import *

In [2]:
help(Focus)

Help on class Focus in module changepoint_online.focus:

class Focus(builtins.object)
 |  Focus(comp_func)
 |  
 |  The Focus class implements the FOCuS method, an algorithm for detecting changes in data streams on one-parameter exponential family models.
 |  For instance, FOCuS can detect changes-in-mean in a Gaussian data stream (white noise). 
 |  It can be applied to settings where either the pre-change parameter is known or unknown.
 |      
 |  FOCuS solves the CUSUM likelihood-ratio test exactly in O(log(n)) time per iteration, where n is the current iteration. 
 |  The method is equivalent to running a rolling window (MOSUM) simultaneously for all sizes of window, or the Page-CUSUM for all possible values
 |  of the size of change (an infinitely dense grid). 
 |  
 |  DISCLAIMER: Albeit the FOCuS algorithm decreases the per-iteration cost from O(n) to O(log(n)), this 
 |  implementation is not technically online as for n->infty this code would inherently overflow. 
 |  True onl

In [3]:
import numpy as np

In [4]:
np.random.seed(0)
Y = np.concatenate((np.random.normal(loc=0.0, scale=1.0, size=5000), np.random.normal(loc=10.0, scale=1.0, size=5000)))

### Guassian

In [6]:
detector = Focus(Gaussian(loc=0))

In [7]:
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break

In [8]:
detector.changepoint()

{'stopping_time': 5001, 'changepoint': 5000}

### Poisson

In [14]:
np.random.seed(0)
Y = np.concatenate((np.random.poisson(lam=4.0, size=5000), np.random.poisson(lam=4.2, size=5000)))

In [15]:
detector = Focus(Poisson())

In [16]:
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break

In [17]:
detector.changepoint()

{'stopping_time': 6488, 'changepoint': 4973}

### Gamma

In [18]:
np.random.seed(0)
Y = np.concatenate((np.random.gamma(4.0, scale=3.0, size=5000), np.random.gamma(4.0, scale=6.0, size=5000)))

In [34]:
detector = Focus(Gamma(scale=3.0, shape=4.0))
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break
detector.changepoint()

{'stopping_time': 5010, 'changepoint': 5002}

In [33]:
# with the Gamma it's possible to initialize even with rate parameter
detector = Focus(Gamma(rate=1/3.0, shape=4.0))
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break
detector.changepoint()

{'stopping_time': 5010, 'changepoint': 5002}

### Bernoulli
Bernoulli seems to be dead.

In [10]:
np.random.seed(0)
Y = np.concatenate((np.random.binomial(n=1, p=.4, size=5000), np.random.binomial(n=1, p=.8, size=5000)))

In [13]:
detector = Focus(Bernoulli())
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break
detector.changepoint()

ValueError: math domain error